In [1]:
import numpy as np
from ase import Atoms
from pymatgen.core import Structure
from pymatgen.core.surface import SlabGenerator
from pymatgen.io.ase import AseAtomsAdaptor
from scipy.spatial.distance import cdist
from ase.io import read
from ase.visualize import view
from ase.db import connect
import nglview

In [2]:
import numpy as np
import math
from ase import Atoms
from pymatgen.core import Structure
from pymatgen.core.surface import SlabGenerator
from pymatgen.io.ase import AseAtomsAdaptor
from scipy.spatial import cKDTree

In [16]:
class SlabWaterAdsorption:
    """
    A class for generating symmetric slabs with adsorbed water molecules.

    This class takes a bulk structure, creates symmetric slabs based on specified Miller indices,
    and adds water molecules to the surface of these slabs.

    Attributes:
        input_structure (Structure): The input bulk structure (Pymatgen Structure or ASE Atoms).
        miller_index (tuple): The Miller index for slab generation.
        min_slab_size (float): Minimum slab size in Angstroms.
        min_vacuum_size (float): Minimum vacuum size in Angstroms.
        _slabs (list): Cached list of generated slabs.

    """

    def __init__(self, input_structure, miller_index, min_slab_size, min_vacuum_size):
        """
        Initialize the SlabWaterAdsorption object.

        Args:
            input_structure (Structure or Atoms): The input bulk structure.
            miller_index (tuple): The Miller index for slab generation.
            min_slab_size (float): Minimum slab size in Angstroms.
            min_vacuum_size (float): Minimum vacuum size in Angstroms.
        """
        self.input_structure = self._convert_to_pymatgen(input_structure)
        self.miller_index = miller_index
        self.min_slab_size = min_slab_size
        self.min_vacuum_size = min_vacuum_size
        self._slabs = None

    def _convert_to_pymatgen(self, input_structure):
        """
        Convert input structure to Pymatgen Structure if necessary.

        Args:
            input_structure (Structure or Atoms): The input structure.

        Returns:
            Structure: The input structure converted to Pymatgen Structure.

        Raises:
            ValueError: If the input structure is neither ASE Atoms nor Pymatgen Structure.
        """
        if isinstance(input_structure, Atoms):
            return AseAtomsAdaptor.get_structure(input_structure)
        elif isinstance(input_structure, Structure):
            return input_structure
        else:
            raise ValueError("Input structure must be either ASE Atoms or Pymatgen Structure")

    def _create_symmetric_slabs(self):
        """
        Create symmetric slabs from the input structure.

        Returns:
            list: A list of symmetric slabs (Pymatgen Slab objects).
        """
        if self._slabs is None:
            slab_gen = SlabGenerator(self.input_structure, self.miller_index, self.min_slab_size, self.min_vacuum_size, center_slab=True,
                                     lll_reduce=True,
                                     max_normal_search=5,
                                     in_unit_planes=True)
            self._slabs = slab_gen.get_slabs(symmetrize=True)
        return self._slabs

    @staticmethod
    def tile_atoms(atoms: Atoms, min_ab: float = 8):
        """
        Tile the atoms to meet a minimum size in the a and b directions.

        Args:
            atoms (Atoms): The input ASE Atoms object.
            min_ab (float): The minimum size in Angstroms for a and b directions.

        Returns:
            Atoms: The tiled ASE Atoms object.
        """
        a_length = np.linalg.norm(atoms.cell[0])
        b_length = np.linalg.norm(atoms.cell[1])
        na = int(math.ceil(min_ab / a_length))
        nb = int(math.ceil(min_ab / b_length))
        n_abc = (na, nb, 1)
        atoms_tiled = atoms.repeat(n_abc)
        return atoms_tiled

    def _add_water_molecules(self, atoms, num_water, min_distance_water, min_surface_distance, max_surface_distance):
        """
        Add water molecules to the surface of the slab.

        Args:
            atoms (Atoms): The input ASE Atoms object (slab).
            num_water (int): Number of water molecules to add.
            min_distance_water (float): Minimum distance between water molecules.
            min_surface_distance (float): Minimum distance from the surface for water placement.
            max_surface_distance (float): Maximum distance from the surface for water placement.

        Returns:
            Atoms: The ASE Atoms object with added water molecules.
        """
        water = Atoms('H2O', positions=[[0, 0, 0], [0.757, 0.586, 0], [0.757, -0.586, 0]])
        cell = atoms.get_cell()
        top_z = max(atoms.positions[:, 2])
        all_positions = atoms.get_positions()

        for _ in range(num_water):
            for _ in range(1000):  # Max attempts per water molecule
                x = np.random.uniform(0, cell[0, 0])
                y = np.random.uniform(0, cell[1, 1])
                z = top_z + np.random.uniform(min_surface_distance, max_surface_distance)

                angle_x = np.random.uniform(-180, 180)
                angle_y = np.random.uniform(-180, 180)
                angle_z = np.random.uniform(-180, 180)
                
                rotated_water = water.copy()
                rotated_water.rotate(angle_x, 'x', center='COM')
                rotated_water.rotate(angle_y, 'y', center='COM')
                rotated_water.rotate(angle_z, 'z', center='COM')

                translated_water = rotated_water.copy()
                translated_water.translate([x, y, z])

                new_positions = translated_water.get_positions()
                
                if self._check_distances(new_positions, all_positions, min_distance_water):
                    atoms.extend(translated_water)
                    all_positions = np.vstack((all_positions, new_positions))
                    break
            else:
                print(f"Warning: Could not place water molecule {_+1} after 1000 attempts.")

        return atoms

    def _check_distances(self, new_positions, existing_positions, min_distance):
        """
        Check if new positions maintain minimum distance from existing positions.

        Args:
            new_positions (np.array): Array of new positions to check.
            existing_positions (np.array): Array of existing positions.
            min_distance (float): Minimum allowed distance between positions.

        Returns:
            bool: True if all new positions maintain minimum distance, False otherwise.
        """
        if len(existing_positions) == 0:
            return True
        
        tree = cKDTree(existing_positions)
        distances, _ = tree.query(new_positions)
        return np.all(distances > min_distance)

    def get_all_slabs_with_water(self, num_water, min_distance_water=2, min_surface_distance=1.2, max_surface_distance=4.0, min_xy_size=8):
        """
        Generate all symmetric slabs with water molecules added to the surface.

        Args:
            num_water (int): Number of water molecules to add to each slab.
            min_distance_water (float): Minimum distance between water molecules.
            min_surface_distance (float): Minimum distance from the surface for water placement.
            max_surface_distance (float): Maximum distance from the surface for water placement.
            min_xy_size (float): Minimum size of the slab in x and y directions.

        Returns:
            list: A list of tuples, each containing:
                - Atoms: The ASE Atoms object of the slab with water.
                - dict: Metadata about the slab and water addition process.

        Raises:
            ValueError: If no symmetric slabs could be generated.
        """
        slabs = self._create_symmetric_slabs()
        if not slabs:
            raise ValueError("No symmetric slabs could be generated.")

        results = []
        for i, slab in enumerate(slabs):
            selected_atoms = AseAtomsAdaptor.get_atoms(slab)

            # Tile the slab
            tiled_atoms = self.tile_atoms(selected_atoms, min_xy_size)

            # Add water molecules to the tiled slab
            final_atoms = self._add_water_molecules(tiled_atoms, num_water, min_distance_water, min_surface_distance, max_surface_distance)

            # Collect metadata
            metadata = self._collect_metadata(i, slab, final_atoms, num_water, min_xy_size, min_surface_distance, max_surface_distance)

            results.append((final_atoms, metadata))

        return results

    def _collect_metadata(self, slab_index, original_slab, final_atoms, num_water, min_xy_size, min_surface_distance, max_surface_distance):
        """
        Collect metadata about the slab and water addition process.

        Args:
            slab_index (int): Index of the slab.
            original_slab (Slab): The original Pymatgen Slab object.
            final_atoms (Atoms): The final ASE Atoms object with water added.
            num_water (int): Number of water molecules added.
            min_xy_size (float): Minimum size of the slab in x and y directions.
            min_surface_distance (float): Minimum distance from the surface for water placement.
            max_surface_distance (float): Maximum distance from the surface for water placement.

        Returns:
            dict: A dictionary containing metadata about the slab and water addition process.
        """
        return {
            "slab_index": slab_index,
            "miller_index": self.miller_index,
            "original_slab_atoms": len(original_slab),
            "final_atoms": len(final_atoms),
            "water_molecules_added": num_water,
            "min_xy_size": min_xy_size,
            "min_surface_distance": min_surface_distance,
            "max_surface_distance": max_surface_distance,
            "final_cell_dimensions": final_atoms.cell.cellpar().tolist(),
            "surface_area": np.linalg.norm(np.cross(final_atoms.cell[0], final_atoms.cell[1]))
        }

    def get_number_of_slabs(self):
        """
        Get the number of symmetric slabs generated.

        Returns:
            int: The number of symmetric slabs.
        """
        return len(self._create_symmetric_slabs())

In [17]:
atoms = read('./bto_cubic.cif')

In [18]:
atoms

Atoms(symbols='BaTiO3', pbc=True, cell=[4.00768164, 4.00768164, 4.00768164], spacegroup_kinds=...)

In [6]:
# db = connect('test.db')

In [19]:
for indx in (1, 1, 1), (1, 1, 0), (1, 0, 0):
    if indx == (1, 0, 0):
        swa = SlabWaterAdsorption(atoms, miller_index=indx, min_slab_size=4, min_vacuum_size=4)
    else:
        swa = SlabWaterAdsorption(atoms, miller_index=indx, min_slab_size=5, min_vacuum_size=5)
    for i in range(100):
        # Initialize the SlabWaterAdsorption object

        # Get all slabs with water molecules added
        results = swa.get_all_slabs_with_water(num_water=4, 
                                            min_distance_water=1.2, 
                                            min_surface_distance=1.2,
                                            max_surface_distance=4.0,
                                            min_xy_size=7)
        ad_atoms = results[0][0]
        # db.write(ad_atoms)

    

In [20]:
results

[(MSONAtoms(symbols='H8Ba12O48Ti16', pbc=True, cell=[[8.01536328, 0.0, 4.907994492427616e-16], [-4.907994492427616e-16, 8.01536328, 4.907994492427616e-16], [0.0, 0.0, 32.06145312]], bulk_equivalent=..., bulk_wyckoff=..., spacegroup_kinds=...),
  {'slab_index': 0,
   'miller_index': (1, 0, 0),
   'original_slab_atoms': 18,
   'final_atoms': 84,
   'water_molecules_added': 4,
   'min_xy_size': 7,
   'min_surface_distance': 1.2,
   'max_surface_distance': 4.0,
   'final_cell_dimensions': [8.01536328,
    8.01536328,
    32.06145312,
    90.0,
    90.0,
    90.0],
   'surface_area': 64.24604851037238}),
 (MSONAtoms(symbols='H8Ba16O44Ti12', pbc=True, cell=[[8.01536328, 0.0, 4.907994492427616e-16], [-4.907994492427616e-16, 8.01536328, 4.907994492427616e-16], [0.0, 0.0, 32.06145312]], bulk_equivalent=..., bulk_wyckoff=..., spacegroup_kinds=...),
  {'slab_index': 1,
   'miller_index': (1, 0, 0),
   'original_slab_atoms': 17,
   'final_atoms': 80,
   'water_molecules_added': 4,
   'min_xy_size'

In [21]:
final_atoms = results[0][0]

In [22]:
final_atoms.cell.cellpar()

array([ 8.01536328,  8.01536328, 32.06145312, 90.        , 90.        ,
       90.        ])

In [23]:
view(final_atoms, viewer='x3d')